## WCS solutions for AuxTel - 24-Nov-22

Craig Lage 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from astroquery.astrometry_net import AstrometryNet

import lsst.afw.image as afwImage
from lsst.summit.utils.blindSolving import runImchar, CommandLineSolver
# This requires to be on the tickets/DM-31175 of summit_utils
from lsst.summit.utils import BestEffortIsr
from lsst.daf.butler import Butler

In [ ]:
solver = CommandLineSolver(indexFiles='/scratch/mfisherlevine/astrometry_net/broiler.astrometry.net/~dstn/4100',
                           checkInParallel=True)
best_effort_isr = BestEffortIsr()

In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib"])
dataId = {'day_obs': 20221025, 'seq_num': 253, 'detector': 0, "instrument": 'LATISS'}
best_effort_isr.doWrite = False  # Don't write to butler database
exp = best_effort_isr.getExposure(dataId, skipCosmics=False)
mData = butler.get('raw.metadata', dataId=dataId)

In [ ]:
# This uses the local solver, but has been failing to solve.
# I don't know why
results = runImchar(exp, 5, 25)
solverResults = solver.run(exp, results.sourceCat, silent=True)


In [ ]:
# So we'll use the remote astrometry.net solver
ast = AstrometryNet()
ast.api_key = 'xxawwhvleirxcswx'

In [ ]:
sourceCat = results.sourceCat
x = sourceCat.getColumnView().getX()
y = sourceCat.getColumnView().getY()
wcs = exp.getWcs()
image_width = int(mData['DETSIZE'].strip('[]').split(',')[0].split(':')[1])
image_height = int(mData['DETSIZE'].strip('[]').split(',')[1].split(':')[1])
scale_units = 'arcsecperpix'
scale_type='ev' # ev means submit estimate and % error
scale_est = wcs.getPixelScale().asArcseconds()
scale_err = 2.0
center_ra = mData['RA']
center_dec = mData['DEC']
radius = 0.5
wcs_anet = ast.solve_from_source_list(x, y, 
                                        image_width, image_height, scale_units=scale_units,
                                        scale_type=scale_type, scale_est=scale_est, scale_err=scale_err,
                                        center_ra=center_ra, center_dec=center_dec, radius=radius,
                                        crpix_center=True, solve_timeout=240)

### Now compare the results


We've constrained a.net to put the pixel origin in the center, but this is not the case for\
the DM wcs.  So we need to find the DM wcs coordinates at the origin, and then compare to \
the a.net solution.

In [ ]:
# Get the DM wcs coordinates of the center pixel:
DM_center = wcs.pixelToSky(wcs_anet['CRPIX1'], wcs_anet['CRPIX2'])
DM_RA_Dec = (DM_center.getRa().asDegrees(), DM_center.getDec().asDegrees())
print(f"DM center pix RA/Dec = {DM_RA_Dec[0]}, {DM_RA_Dec[1]} degrees")
Anet_RA_Dec = (wcs_anet['CRVAL1'], wcs_anet['CRVAL2'])
print(f"Anet center pix RA/Dec = {Anet_RA_Dec[0]}, {Anet_RA_Dec[1]} degrees")
Diff = ((DM_RA_Dec[0] - Anet_RA_Dec[0]) * 3600.0, (DM_RA_Dec[1] - Anet_RA_Dec[1]) * 3600.0)
print(f"Difference center pix RA/Dec = {Diff[0]}, {Diff[1]} arcseconds")

In [ ]:
# Now look at the data with matplotlib
# This shows which objects were used by a.net
%matplotlib inline
from matplotlib.colors import LogNorm

def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.suptitle(f"Image",fontsize=18)
arr = exp.image.array
img = plt.imshow(arr, vmin=1, vmax=20, interpolation='Nearest', cmap='gray')
colorbar(img)
plt.tight_layout(h_pad=1)
plt.scatter(x,y,facecolors='none', edgecolors='y', s=200, lw=2)